**Data Preprocessing**

In [32]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import os
import pandas as pd
import random
from skimage import io
from shutil import copyfile
import sys
import time
import json

import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [33]:
original_fname = 'malaria2ndhalf.csv'
data_df = pd.read_csv(original_fname)
data_df.shape
data_df.head()

,image,id,label,annotator,annotation_id,created_at,updated_at,lead_time
0,/data/upload/1/f25e92ca-Trip_065_Day_2_01-12-0...,150,"[{""x"": 28.571428571428577, ""y"": 0, ""width"": 5....",2,152,2022-11-02T11:59:56.537652Z,2022-11-02T11:59:56.537652Z,279.229
1,/data/upload/1/a732c3d5-Trip_065_Day_2_01-12-0...,149,"[{""x"": 7.263922518159806, ""y"": 14.610389610389...",2,151,2022-11-02T11:55:15.904090Z,2022-11-02T11:55:15.904090Z,242.082
2,/data/upload/1/a75d2e1c-Trip_065_Day_2_01-12-0...,148,"[{""x"": 18.401937046004843, ""y"": 15.90909090909...",2,150,2022-11-02T11:51:11.318863Z,2022-11-02T11:51:11.318863Z,219.055
3,/data/upload/1/1d69f1e6-Trip_065_Day_2_01-12-0...,147,"[{""x"": 3.8740920096852296, ""y"": 53.57142857142...",2,149,2022-11-02T11:47:31.235369Z,2022-11-02T11:47:31.235369Z,69.525
4,/data/upload/1/ad33c564-Trip_065_Day_2_01-12-0...,146,"[{""x"": 0, ""y"": 26.948051948051948, ""width"": 3....",2,148,2022-11-02T11:44:10.383394Z,2022-11-02T11:44:10.383394Z,267.925


In [69]:
new_df = pd.DataFrame(columns=['FileName', 'X1', 'Y1', 'X2', 'Y2', 'ClassName'])
for i in range(data_df.shape[0]):
    img_name = (data_df['image'].iloc[i])[15:]
    labels = (data_df['label'].iloc[i])[1:-1]
    labels = labels.split('},')
    num_labels = len(labels)
    #print(num_labels)
    for label in labels:
        if label[-1] != '}':
            label = label+'}'
        res = json.loads(label)
        res['x'] = int(res['x']/100*res['original_width'])
        res['y'] = int(res['y']/100*res['original_height'])
        res['width'] = int(res['width']/100*res['original_width'])
        res['height'] = int(res['height']/100*res['original_height'])
        #print(res['x'],res['y'],res['width'],res['height'])
        new_df = new_df.append({'FileName': img_name, 
                                            'X1': res['x'], 
                                            'Y1': res['y'], 
                                            'X2': res['x']+res['width'], 
                                            'Y2': res['y']+res['height'], 
                                            'ClassName': res['rectanglelabels'][0]}, 
                                           ignore_index=True)
new_df.shape
new_df.head()

,FileName,X1,Y1,X2,Y2,ClassName
0,f25e92ca-Trip_065_Day_2_01-12-05_Image_7_3.png,394,0,470,60,Esch
1,f25e92ca-Trip_065_Day_2_01-12-05_Image_7_3.png,337,334,367,370,DEBRIS
2,f25e92ca-Trip_065_Day_2_01-12-05_Image_7_3.png,157,461,193,494,DEBRIS
3,f25e92ca-Trip_065_Day_2_01-12-05_Image_7_3.png,294,438,354,491,LT
4,f25e92ca-Trip_065_Day_2_01-12-05_Image_7_3.png,371,608,451,671,Esch
5,f25e92ca-Trip_065_Day_2_01-12-05_Image_7_3.png,558,986,665,1029,Esch
6,f25e92ca-Trip_065_Day_2_01-12-05_Image_7_3.png,766,608,846,664,LT
7,f25e92ca-Trip_065_Day_2_01-12-05_Image_7_3.png,836,531,933,607,Lsch
8,f25e92ca-Trip_065_Day_2_01-12-05_Image_7_3.png,1301,672,1327,698,R
9,f25e92ca-Trip_065_Day_2_01-12-05_Image_7_3.png,913,250,976,300,DEBRIS


In [72]:
new_df.to_csv('processed.csv')
# for training
with open("annotation.txt", "w+") as f:
    for i in range(new_df.shape[0]):
        fileName = new_df['FileName'].iloc[i]
        x1 = new_df['X1'].iloc[i]
        y1 = new_df['Y1'].iloc[i]
        x2 = new_df['X2'].iloc[i]
        y2 = new_df['Y2'].iloc[i]
        className = new_df['ClassName'].iloc[i]
        f.write(fileName + ',' + str(x1) + ',' + str(y1) + ',' + str(x2) + ',' + str(y2) + ',' + className + '\n')